# Reinforcement Learning Code, Part 2: Agents that act


Here we are going to implement a RL agent interacting with a simple environment. In this case, our agent would be a simulated RL mouse, and the environment a 2D maze.

## Defining the environment

To define the environment, we need to define the set of possible states $S = {s_1, s_2 ... s_N}$, the transition function $P_{s,s'}^{a}$, and the reward transition function $R_{s,s'}^{a}$.

In our case, the environment just consists of a 4x4 grid. Our hypothetical agent perceives only one cell at any time---the cell where it is.  Therefore, our states correspond to the sixteen position of the maze, which we can indicate with the coordinates $(0, 0), (0, 1)... (0, 3), (1, 0) ... (3, 3)$.

An environment is characterized by two functions:

* The state transition probability function $P(s, a, s')$, which the probability of transitioning to a possible state $s'$ when action $a$ is applied during state $s$; and

* The reward transition probability function $R(s, a, s')$, which is the probability of receiving a reward $r$ when action $a$ is applied to state $s$ and the environment moves to state $s'

In our simple cases, both $P(s,a,s')$ and $R(s,a,s')$ will be simplified to deterministic functions.


In [ ]:
import random
import numpy as np
from copy import copy
import matplotlib.pyplot as plt 

class Maze():
    """A maze environment"""

    ACTIONS = ("up", "down", "left", "right") # List of actions
    INITIAL_STATE = (0, 0) # Always starts at the topleft corner
    
    def __init__(self, fname = "grid.txt"):
        """Inits a maze by loading the grid file"""
        self.grid = np.loadtxt(fname)
        self.state = self.INITIAL_STATE
        self.end = False


    def state_transition(self, state1, action1):
        "Defines the next state gien the "
        s = state1
        state2 = copy(state1)
        
        if action1 in self.ACTIONS:
            if action1 == "up":
                if s[0] > 0:
                    state2 = (s[0] - 1, s[1])
            
            elif action1 == "left":
                if s[1] > 0:
                    state2 = (s[0], s[1] - 1)
            
            elif action1 == "down":
                if s[0] < (self.grid.shape[0] - 1):
                    state2 = (s[0] + 1, s[1])

            elif action1 == "right":
                if s[1] < (self.grid.shape[1] - 1):
                    state2 = (s[0], s[1] + 1)
                    
        return state2
                    
    
    def reward_transition(self, state1, action1, state2):
        """Reward is -1 for bouncing against the walls, and whatever is on the grid otherwise"""
        if state1 == state2:
            return -1
        else:
            return self.grid[state2[0], state2[1]]
        
    
    # Quick way to combine State transitions and Reward transitions 
    def transition(self, action1):
        """Changes the state following an action"""
        state1 = self.state
        state2 = self.state_transition(state1, action1)
        reward2 = self.reward_transition(state1, action1, state2)
        
        self.state = state2
        return (state2, reward2) # Returns s_t+1, r_t+1

    
    def print_state(self):
        "Prints a text representation of the maze (with the agent position)"
        bar = "-" * ( 4 * self.grid.shape[1] + 1)
        for i in range(self.grid.shape[0]):
            row = "|"
            for j in range(self.grid.shape[1]):
                cell = " "
                if i == self.state[0] and j == self.state[1]:
                    cell = "*"
                row += (" %s |" % cell)
            print(bar)
            print(row)
        print(bar)

## Testing the maze
The maze is simple but functional. It is easy to create a maze, check the available actions, apply a few actions, and so on. Let's check that our environment works.

In [ ]:
m = Maze()
m.print_state()
state_after_bad_action = m.state_transition(m.state, "up")  # Illigal action: bounces!
state_after_good_action = m.state_transition(m.state, "down") # Legal action: Goes down

print("State after illegal action: %s" % (state_after_bad_action,))
print("Reward after illegal action: %s" % (m.reward_transition(m.state, "up", state_after_bad_action)))

print("State after legal action: %s" % (state_after_good_action,))
print("Reward after legal action: %s" % (m.reward_transition(m.state, "down", state_after_good_action)))



And, finally, we can easily navigate in our virtual maze by executing the appropriate actions:

In [ ]:
m.transition("down")

Note how the ```Maze``` object returns two values at the end of each action execution, the new state $s_{t+1}$ and the associated reward $r_{t+1}$. If the ```down``` action was executed with the original maze layout of the ```grid.txt``` file, case, the two values are $s_{t+1} = $ ```(3, 0)``` and $r_{t+1} = $ ```0.0```. We can also execute more actions, and see what happens after a few movements:

In [ ]:
m.transition("down")
m.transition("down")
m.transition("right")
m.print_state()


## Creating a Q-Agent
Now we can create our own very fantastic agents! As an example, we will create a $Q$-learning agent that interacts with the ``` Maze``` world. The $Q$-table should contain $16 \times 4 = 64$ different state-action pairs; rather than filling it up right away, we will fill it up as we encounter new state-action pairs, assuming that any previously unencountered state has a value of $Q = 0$ 

In [ ]:
import random
import numpy as np

class Agent():
    def __init__(self, actions=Maze.ACTIONS, epsilon=0.1, alpha=0.1, gamma=0.9):
        """Creates a Q-agent"""
        self.Q = {}    ## Initial dictionary of (s, a) pairs. At the beginning, it's emtpy.

        self.epsilon = epsilon     # Epsilon for e-greey policy
        self.alpha = alpha         # Learning rate
        self.gamma = gamma         # Temporal discounting
        self.actions = actions     # Set of possible actions (provide those of Maze.ACTIONS)


    def policy(self, state):
        """Selects an action with a epsilon-greedy policy"""
        if random.random() < self.epsilon:
            action = random.choice(self.actions)
        else:
            q = [self.Q[(state, a)] if (state, a) in self.Q.keys() else 0.0 for a in self.actions]
            maxQ = max(q)
            count = q.count(maxQ)
            if count > 1:
                best = [i for i in range(len(self.actions)) if q[i] == maxQ]
                i = random.choice(best)
            else:
                i = q.index(maxQ)

            action = self.actions[i]
        return action

    
    def learnQ(self, state1, action1, reward2, state2, action2):
        """Updates the Q-values when given an (s,a) pair, the reward value and a new state"""
        g = self.gamma
        a = self.alpha
        
        q1 = 0.0
        
        if (state1, action1) in self.Q.keys():
            q1 = self.Q[(state1, action1)]
        
        max_q2 = max([self.Q[(state2, a)] if (state2, a) in self.Q.keys() else 0.0 for a in self.actions])
        
        rpe = reward2 + g * max_q2 - q1
        #print("s1 = %s, a1 = %s, s2 = %s, q1 = %.4f, r2 = %.4f, q2 = %.4f" % (state1, action1, state2, q1, reward2, max_q2))
        q1 += a * rpe
        #print("new q1 = %.4f" % (q1,))
        self.Q[(state1, action1)] = q1

            
    def calculateV(self, task):
        """Returns a representation of the best Q values in every states"""
        v = np.zeros(task.grid.shape)
        for i in range(v.shape[0]):
            for j in range(v.shape[1]):
                maxq = max([self.getQ((i, j), a) for a in self.actions])
                v[i,j] =maxq
        return v
    
    def visualizeQ(self):
        """Visualizes the Q tables, one per action"""
        for a in self.actions:
            # Create the corresponding state table
            mat = np.zeros((4,4))
            states = [x for x in self.Q.keys() if x[1] == a]
            
            for s in states:
                row, col = s[0]
                mat[row, col] = self.Q[s]
            
            # Show the Q-table as a heatmap
            plt.imshow(mat, interpolation = "none", vmin=-1, vmax=10, cmap='jet')
            plt.title("Q-Table for action '%s'" % a.upper())
            plt.show()

        

## Interaction between Environment and Agent
Now, two functions to run multiple trials.

In [ ]:
def run_trial(environment, agent):
    """A trial ends when the agent gets a reward. The history is returned"""
    state1 = environment.state
    action1 = agent.policy(state1)
    reward2 = 0.0
    history = [state1]
    while reward2 != 10:
        state2, reward2 = environment.transition(action1)
        action2 = agent.policy(state2)
        
        # Save the states visited
        history.append(state2)
        
        # Update the Q-values for state1, action1
        agent.learnQ(state1, action1, reward2, state2, action2)
        
        state1 = state2
        action1 = action2
        
    return history

    
def run_trials(environment, agent, n, collect=True):
    """Runs N trials"""
    history = []
    for j in range(n):
        h = run_trial(environment, agent)
        history += h
        environment.state = Maze.INITIAL_STATE
    
    return history
        
def visualize_history(history):
    mat = np.zeros((4,4))            
    for s in history:
        row, col = s
        mat[row, col] += 1
    
    mat /= np.sum(mat)
    mat *= 5  # min number of moves
    
    plt.imshow(mat, interpolation = "none", vmin=0, vmax=1, cmap='jet')
    plt.title("Preferred path")
    plt.show()
    

## Testing the agent

Here a few tests.

In [ ]:
m = Maze()
a = Agent()
run_trials(m, a, 10000)
a.visualizeQ()
h = run_trials(m, a, 10)
visualize_history(h)